In [1]:
import pandas as pd
import numpy as np
import json, random

In [29]:
org_df = pd.read_excel('Dump data 1.1.xlsx',
                       sheet_name='org data sheet')

In [30]:
org_df.dropna(subset=['Unique_ employee_ ID'],inplace=True)

In [31]:
def juniors(sub_df):
    return list(sub_df.T.to_dict().values())

org_agg_df = org_df.groupby('Supervisor_ Unique_ ID').apply(juniors)

In [32]:
org_agg_df = org_agg_df.reset_index()

org_agg_df.columns = ['Unique_ employee_ ID','Subordinates']

In [33]:
org_fin_df = pd.merge(org_df,org_agg_df,on='Unique_ employee_ ID',how='left')

In [39]:
org_dct = {}

In [35]:
boss_id = org_fin_df.loc[org_fin_df['Supervisor_ Unique_ ID'].isna(),
'Unique_ employee_ ID'].values[0]

In [40]:
org_dct['name'] = org_fin_df.loc[org_fin_df['Unique_ employee_ ID']
== boss_id,'Employee_ name'].values[0]

In [37]:
org_dct['title'] = org_fin_df.loc[org_fin_df['Unique_ employee_ ID']
== boss_id,'Role_ Title'].values[0]

In [41]:
org_dct['children'] = []

In [28]:
boss_id = 1000246965.00000
org_fin_df.loc[org_fin_df['Unique_ employee_ ID']
== boss_id, 'Loaded_ Cost'].values[0]

500000.0

In [42]:
def nested_children(boss_id,children_list=[]):
    global org_fin_df
    children = org_fin_df.loc[org_fin_df['Unique_ employee_ ID']
== boss_id, 'Subordinates'].tolist()[0]
    if type(children)!=float:
        for child in children:
            child_dct = {}
            id = child['Unique_ employee_ ID']
            child_dct['name'] = child['Employee_ name']
            # child_dct['title'] = child['Role_ Title']
            if nested_children(id,[]):
                child_dct['children'] = nested_children(id,[])
            else:
                child_dct['value'] = org_fin_df.loc[org_fin_df['Unique_ employee_ ID']
== boss_id, 'Loaded_ Cost'].values[0]
            children_list.append(child_dct)
        return children_list
    else:
        return children_list

In [43]:
org_dct['children'] = nested_children(boss_id)

In [80]:
children = org_fin_df.loc[org_fin_df['Unique_ employee_ ID']
== boss_id, 'Subordinates'].tolist()[0]

In [44]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, list):
          for dct in value:
             pretty(dct, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [23]:
pretty(org_dct)


name
	Val oswalt
children
	name
		Stan Polomoski
	children
		name
			MC Regan
		value
			16695
	name
		Brad Troup
	children
		name
			Shane Sagers
		children
			name
				Rashaad Clavon
			value
				19430
		name
			Chika Ogburia
		value
			8712
		name
			Mike Sommers
		value
			17676
		name
			Mike Ross
		value
			14483
		name
			Jennifer Silva
		children
			name
				Andrew Alikhani
			children
				name
					Kristin Brigman
				value
					19401
				name
					Connie Thomas
				value
					10533
				name
					Jeanne Martin
				value
					11130
			name
				Stephen Bell
			children
				name
					Sean Grune
				value
					8197
				name
					Jaclyn Marchigiani
				value
					15914
				name
					Shawn Burkett
				value
					6061
			name
				Andrew Hackel
			children
				name
					Jackson Kiligerman (OPEN)
				value
					13751
				name
					Micheal Picone
				value
					4108
			name
				Trey Mask
			children
				name
					Chriss Ball
				children
					name
						Sean Dooher
					value
						3602

In [45]:
with open('org_data_sunburst.json', 'w', encoding='utf-8') as f:
    json.dump(org_dct, f, ensure_ascii=False, indent=4)